<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Homework9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PREFIX="https://github.com/alexeygrigorev/large-datasets/releases/download/hairstyle"
DATA_URL=f"{PREFIX}/hair_classifier_v1.onnx.data"
MODEL_URL=f"{PREFIX}/hair_classifier_v1.onnx"
!wget $DATA_URL
!wget $MODEL_URL

In [2]:
pip install pillow

In [3]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [4]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 99.8 MB/s eta 0:00:00


In [ ]:
import onnx

# Load your ONNX model
model = onnx.load("/content/hair_classifier_v1.onnx")

# Get the output names
output_names = [output.name for output in model.graph.output]

print("Output names:", output_names)


Output names: ['output']


In [5]:
!wget 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'

--2025-12-04 10:04:45--  https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg
Resolving habrastorage.org (habrastorage.org)... 95.47.173.34, 95.47.173.35, 2a14:b680:0:56::34, ...
Connecting to habrastorage.org (habrastorage.org)|95.47.173.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 398272 (389K) [image/jpeg]
Saving to: ‘yf_dokzqy3vcritme8ggnzqlvwa.jpeg’

yf_dokzqy3vcritme8g 100%[===================>] 388.94K   807KB/s    in 0.5s    

2025-12-04 10:04:47 (807 KB/s) - ‘yf_dokzqy3vcritme8ggnzqlvwa.jpeg’ saved [398272/398272]



In [31]:
img = prepare_image(download_image('https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'),(200,200))

In [32]:
x = img_to_array(img)                         # NumPy array, float32 [web:8]
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
x[0,0,0]

array([-81.939   , -12.778999, -62.68    ], dtype=float32)

In [33]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [34]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 1) Conv layer: in 3 ch, out 32 ch, kernel 3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        # 2) Max pooling: 2x2
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        # After conv1 (no padding): input (3,200,200) -> (32, 198, 198)
        # After maxpool 2x2: -> (32, 99, 99)
        # Flatten size = 32 * 99 * 99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()  # binary output

    def forward(self, x):
        # x: (batch, 3, 200, 200)
        x = self.conv1(x)          # (batch, 32, 198, 198)
        x = F.relu(x)
        x = self.pool(x)           # (batch, 32, 99, 99)
        x = torch.flatten(x, 1)    # (batch, 32*99*99)
        x = self.fc1(x)            # (batch, 64)
        x = F.relu(x)
        x = self.fc2(x)            # (batch, 1)
        x = self.sigmoid(x)        # probability for class 1
        return x

In [35]:
model = SimpleCNN()

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

criterion = nn.MSELoss()

In [37]:
from PIL import Image
import torch
from torchvision import transforms

# 1) Define same size, channels-first, and normalization if you want
transform = transforms.Compose([
    transforms.Resize((200, 200)),     # model expects 200x200 [web:35]
    transforms.ToTensor(),             # -> (C, H, W), values in [0, 1] [web:36]
    # Optional: add Normalize(...) here if you trained with it [web:39]
])

# 2) Load and transform image
img = img.convert("RGB")  # 3 channels [web:37]
x = transform(img)            # shape (3, 200, 200)
x = x.unsqueeze(0)            # shape (1, 3, 200, 200) add batch dim [web:35]

In [39]:
model = SimpleCNN()
model.eval()                                                            # eval mode [web:37]

with torch.no_grad():
    y = model(x)              # shape (1, 1)
    prob = y.item()           # scalar probability for class 1

# For binary decision:
pred = 1 if prob >= 0.5 else 0
print("prob:", prob, "pred:", pred)

prob: 0.4599649906158447 pred: 0
